<a href="https://colab.research.google.com/github/mishra-atul5001/Data-Science-and-ML-insights-Projects/blob/master/Copy_of_toxic_clasification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from tensorflow.keras import layers,models
#import pyautogui
import nltk
import re
from keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding
from keras.preprocessing import sequence
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling1D,Dense, LSTM,GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [2]:
from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:

data = pd.read_csv("/content/drive/My Drive/toxic_train_2.csv",encoding = 'ISO-8859-1')

data.head(5)

,comment_text,toxic
0,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
1,"""\nFair use rationale for Image:Wonju.jpg\n\nT...",0
2,bbq \n\nbe a man and lets discuss it-maybe ove...,0
3,Hey... what is it..\n@ | talk .\nWhat is it......,1
4,Before you start throwing accusations and warn...,0


In [4]:
data.toxic.value_counts()

0    144111
1     40307
Name: toxic, dtype: int64

In [5]:
ind=data[data['toxic']==0].index.tolist()
ind1=data[data['toxic']==1].index.tolist()
data1=data.iloc[ind[0:40307],:]
data2=data.iloc[ind1[0:],:]

In [6]:
final=data2.append(data1)
final['toxic'].value_counts()

1    40307
0    40307
Name: toxic, dtype: int64

In [7]:
x=final.iloc[:,0].values
y=final['toxic'].values
print(np.shape(x),np.shape(y))

(80614,) (80614,)


#### As shown above, the "toxic" label has 2 prediction classes - "0": negative/non-toxic, "1": positive/toxic.
#### It indicates that training data is highly imbalanced on the prediction classes.

In [8]:
#lowercase letters
for i in range(0,len(x)):
    text=x[i]
    x[i]=text.lower()

#### Remove Contractions

In [9]:
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"i'd": "i had / i would",
"i'd've": "i would have",
"i'll": "i shall / i will",
"i'll've": "i shall have / i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"}
for i in range(0,len(x)):
    for j in range(0,len(contractions)):
        keys=""+list(contractions.keys())[j]
        value=""+list(contractions.values())[j]
        if keys in x[i]:
            phrase = re.sub(keys, value, x[i])
            x[i]=phrase
        else:
            x[i]=x[i]

#### Remove Punctuations

In [10]:
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
for i in range(0,len(x)):
    no_punct = ""
    for char in x[i]:
        if char not in punctuations:
            no_punct = no_punct + char
    x[i]=no_punct

#### Remove Numbers

In [11]:
def remove(list): 
    pattern = '[0-9]'
    list = [re.sub(pattern, '', i) for i in x] 
    return list
x=remove(x)

#### Remove stopwords

In [12]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stopwords=stopwords.words('english')
for i in range(0,len(x)):
    word_tokens = word_tokenize(x[i]) 
    sent=""
    for char in word_tokens:
        if char not in stopwords:
            sent=sent+" "+char
    x[i]=sent
            
        

#### Remove extra spaces 

In [14]:
for i in range(0,len(x)):
    x[i]=" ".join(x[i].split())

In [15]:
#### Tokenize

In [16]:
x[0]
cleaned_train = x.copy()
cleaned_test = y.copy()

In [17]:
cleaned_test[4]


1

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
cv  = CountVectorizer(binary = True) #We put TRUE, because we want all non zero to be 1 as we are looking for Binary Outcomes!
cv.fit(cleaned_train)
X = cv.transform(cleaned_train)
#X_test = cv.transform(cleaned_test)
cleaned_test = np.array(cleaned_test).reshape(-1, 1).ravel()

In [19]:
from sklearn.metrics import accuracy_score
X_train,X_val,y_train,y_val = train_test_split(X,cleaned_test,train_size = 0.7,random_state = 4)
for c in [0.001,0.01,0.05,0.25,0.5,0.75,1]:
    lr = LogisticRegression(C = c,max_iter = 300)
    lr.fit(X_train,y_train)
    print('For C: ', c , 'Accuracy Score: ',round(accuracy_score(y_val,lr.predict(X_val)),4))

For C:  0.001 Accuracy Score:  0.8555
For C:  0.01 Accuracy Score:  0.9118
For C:  0.05 Accuracy Score:  0.9238
For C:  0.25 Accuracy Score:  0.9327
For C:  0.5 Accuracy Score:  0.9347
For C:  0.75 Accuracy Score:  0.9359
For C:  1 Accuracy Score:  0.9364


In [20]:
from sklearn.metrics import confusion_matrix
# Now let's check for the test set
test_model = LogisticRegression(C = 1,max_iter=1000)
test_model.fit(X_train,y_train)
print('Final Accuracy: ',round(accuracy_score(y_val,test_model.predict(X_val)),2))

Final Accuracy:  0.94


In [21]:
#y_test_pred=model.predict_classes(x_test)
num=2490
print('Actual:',y_val[num])

x_test1=np.reshape(X_val[num],(1,132390))

y_test_pred=test_model.predict(x_test1)
print('predicted:',y_test_pred)

Actual: 0
predicted: [0]


In [22]:
x_test1[0]

<1x132390 sparse matrix of type '<class 'numpy.int64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [23]:
#feature_to_coefficient = {word: coef for word,coef in zip(cv.get_feature_names(),test_model.coef_[0])}
#feature_to_coefficient

In [24]:
feature_to_coefficient = {word: coef for word,coef in zip(cv.get_feature_names(),test_model.coef_[0])}

print('Top 5 Toxic Words are: ')
for toxic_words in sorted(feature_to_coefficient.items(),key = lambda x: x[1],reverse = True)[:5]:
    print(toxic_words)

print('------***********------')

print('Top 5 Non Toxic Words are: ')
for non_toxic_words in sorted(feature_to_coefficient.items(),key = lambda x: x[1])[:5]:
    print(non_toxic_words)

Top 5 Toxic Words are: 
('bitch', 6.446801173981827)
('fuck', 5.9928607921307275)
('bitches', 5.853093221524549)
('hoes', 5.5876913605179)
('fucking', 5.523420721826939)
------***********------
Top 5 Non Toxic Words are: 
('redirect', -2.4810725848218307)
('ani', -1.9891536768571911)
('mailbox', -1.5698647597105921)
('apologize', -1.536082963524677)
('talkâ', -1.4696822309316884)


In [25]:
x[1]

'hey | talk exclusive group wp talibanswho good destroying selfappointed purist gang one asks questions abt antisocial destructive noncontribution wp ask sityush clean behavior issue nonsensical warnings'

In [26]:
ngram_cv = CountVectorizer(binary=True,ngram_range=(1,2))
ngram_cv.fit(cleaned_train)
X = ngram_cv.transform(cleaned_train)
#X_test = ngram_cv.transform(cleaned_test_data_from_stop_words)
#cleaned_test = np.array(cleaned_test).reshape(-1, 1).ravel()

In [27]:
X_train,X_val,y_train,y_val = train_test_split(X,cleaned_test,train_size = 0.7,random_state = 4)
for c in [0.001,0.01,0.05,0.25,0.5,0.75,1]:
    lr = LogisticRegression(C = c,max_iter = 300)
    lr.fit(X_train,y_train)
    print('For C: ', c , 'Accuracy Score: ',round(accuracy_score(y_val,lr.predict(X_val)),4))

For C:  0.001 Accuracy Score:  0.8557
For C:  0.01 Accuracy Score:  0.9124
For C:  0.05 Accuracy Score:  0.9251
For C:  0.25 Accuracy Score:  0.9328
For C:  0.5 Accuracy Score:  0.9346
For C:  0.75 Accuracy Score:  0.9355
For C:  1 Accuracy Score:  0.936


In [28]:
# Now let's check for the test set
test_model = LogisticRegression(C = 1,max_iter=1000)
test_model.fit(X_train,y_train)
print('Final Accuracy: ',round(accuracy_score(y_val,test_model.predict(X_val)),2))

Final Accuracy:  0.94


In [29]:
#x1='holy shit'
#test_model.predict(x1)

In [30]:
def show_me_top_words(model):
    feature_to_coefficient = {word: coef for word,coef in zip(cv.get_feature_names(),model.coef_[0])}

    print('Top 5 Toxic Words are: ')
    for toxic_words in sorted(feature_to_coefficient.items(),key = lambda x: x[1],reverse = True)[:5]:
        print(toxic_words)

    print('------***********------')

    print('Top 5 Non Toxic Words are: ')
    for non_toxic_words in sorted(feature_to_coefficient.items(),key = lambda x: x[1])[:5]:
        print(non_toxic_words)

In [31]:
show_me_top_words(test_model)

Top 5 Toxic Words are: 
('worldvieweg', 6.461775729200974)
('yoghurt', 5.5856688165830395)
('outcrossing', 3.8626658882875766)
('oddball', 3.8547120539757556)
('wikimediaprivacy', 3.6311075130655106)
------***********------
Top 5 Non Toxic Words are: 
('irrelvance', -1.4400484707651295)
('httpenwikipediaorgwikishimingyicitenote', -1.2541835011585512)
('kanpur', -1.0979827981093941)
('delete', -1.0688916717574548)
('larxenes', -0.9735541701618435)


In [43]:
#y_test_pred=model.predict_classes(x_test)
num=24180
print('Actual:',y_val[num])

x_test1=np.reshape(X_val[num],(1,1306759))

y_test_pred=test_model.predict(x_test1)
print('predicted:',y_test_pred)

Actual: 1
predicted: [1]


In [57]:
#x1='holy shit'
#test_model.predict(instance)
#x1=np.reshape(x1,(-1,1))
#custom_pred = test_model.predict(datapoint,features = ['x1'])
#print('predicted:',custom_pred)